<a href="https://colab.research.google.com/github/Bhavanish19/Financial_Rag_Model/blob/main/genai_finance_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Financial GenAI Chatbot**

**Problem Statment:** Building a Financial GenAI Base Chatbot for answering Company Specific product queries and helping customer to use and buy products effectively.

**Project Methodology**
- This Project using the Open Source Data of Company Products information with their QNA data.
- Using Python, that load data and then pre-processed and saved in CSV File.
- Loading that same CSV file to insert into Vector DB using Embedding Model from Hugging Face.
- Building RAG QA Chain using Langchain and building the RAG architecture using Zypher 7B LLM (Open Source).
- Checking the Response if Chatbot will able to answer queries effectively.


# Building the GenAI Finance Chatbot

In [1]:
!pip install langchain
!pip install langchain_community
!pip install sentence-transformers
!pip install chromadb
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 943.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata 

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

### Loading the Data

In [4]:
bank = pd.read_csv("https://raw.githubusercontent.com/Bhavanish19/Financial_Rag_Model/main/BankFAQs.csv")
bank.head()

,Question,Answer,Class
0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...,security
1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ...",security
2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...,security
3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...,security
4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...,security


In [5]:
bank["content"] = bank.apply(lambda row: f"Question: {row['Question']}\nAnswer: {row['Answer']}", axis=1)

In [6]:
bank.head()

,Question,Answer,Class,content
0,Do I need to enter ‘#’ after keying in my Card...,Please listen to the recorded message and foll...,security,Question: Do I need to enter ‘#’ after keying ...
1,What details are required when I want to perfo...,"To perform a secure IVR transaction, you will ...",security,Question: What details are required when I wan...
2,How should I get the IVR Password if I hold a...,An IVR password can be requested only from the...,security,Question: How should I get the IVR Password i...
3,How do I register my Mobile number for IVR Pas...,Please call our Customer Service Centre and en...,security,Question: How do I register my Mobile number f...
4,How can I obtain an IVR Password,By Sending SMS request: Send an SMS 'PWD<space...,security,Question: How can I obtain an IVR Password \nA...


In [7]:
from langchain.docstore.document import Document

# Prepare documents for LangChain
documents = []
for _, row in bank.iterrows():
    documents.append(Document(page_content=row["content"], metadata={"class": row["Class"]}))

In [8]:
documents[1]

Document(metadata={'class': 'security'}, page_content='Question: What details are required when I want to perform a secure IVR transaction\nAnswer: To perform a secure IVR transaction, you will need your 16-digit Card number, Card expiry date, CVV number, mobile number and IVR password.')

### Loading Data into Chroma DB

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hg_embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
!pip install chromadb

In [11]:
from langchain.vectorstores import Chroma

persist_directory = '/kaggle/working/'

langchain_chroma = Chroma.from_documents(
    documents=documents,
    collection_name="chatbot_finance",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)

### Loading the Zypher LLM

In [12]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

model_id = 'HuggingFaceH4/zephyr-7b-beta'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

cuda:0


In [13]:
!pip install accelerate

In [14]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

### Building Hugging Face Pipeline to Build LLM Function

In [15]:
# Initialize the query pipeline with increased max_length
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=6000,  # Increase max_length
    max_new_tokens=500,  # Control the number of new tokens generated
    device_map="auto",
)

### Testing the LLM

In [16]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

llm = HuggingFacePipeline(pipeline=query_pipeline)

question = "What is Chatbot and How it used in Finance Domain?"
response = llm(prompt=question)

full_response =  f"Question: {question}\nAnswer: {response}"
display(Markdown(colorize_text(full_response)))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strat



**<font color='red'>Question:</font>** What is Chatbot and How it used in Finance Domain?


**<font color='green'>Answer:</font>** What is Chatbot and How it used in Finance Domain?

A chatbot is an artificial intelligence (AI) program that can simulate a conversation with a human user through text or voice commands. It uses natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots are becoming increasingly popular in the finance domain due to their ability to provide quick and accurate responses to customer inquiries, reducing the need for human intervention.

In the finance domain, chatbots are used for various purposes, such as:

1. Customer Service: Chatbots can provide quick and accurate responses to customer inquiries, such as account balances, transaction history, and bill payments. This reduces the workload on customer service representatives and provides a better customer experience.

2. Financial Advice: Chatbots can provide personalized financial advice based on a user's financial goals, risk tolerance, and investment history. This can help users make informed investment decisions and manage their finances more effectively.

3. Fraud Detection: Chatbots can analyze transaction data in real-time and detect any suspicious activity, such as unusual spending patterns or unauthorized access. This can help prevent fraud and protect user accounts.

4. Personal Finance Management: Chatbots can help users manage their personal finances by providing budgeting advice, tracking expenses, and suggesting ways to save money. This can help users achieve their financial goals and improve their overall financial health.

In summary, chatbots are a powerful tool in the finance domain, providing quick and accurate responses, personalized financial advice, fraud detection, and personal finance management. As AI and NLP technologies continue to advance, chatbots will become even more sophisticated and provide even more value to users in the finance domain.

### Building the RAG QA Chain using Langchain and Create Chatbot Interface

In [17]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from IPython.display import display, Markdown
import os
import warnings
warnings.filterwarnings('ignore')

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

# Define the prompt template
template = """
You are a Finance QNA Expert, Analyze the Query and Respond to Customer with suitable answer. If you don't know the answer, just say "Sorry, I don't know."
Question: {question}
Context: {context}
Answer:
"""
PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)

retriever = langchain_chroma.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

def chat_with_rag():
    print("Welcome to the GenAI Financial Chatbot. Type 'exit' to end the conversation.")
    while True:
        query = input("You: ")
        if query.lower() in ["exit", "quit"]:
            break
        context = "Your context here"
        try:
            result = qa_chain({"context": context, "query": query})
            print(f"Chatbot: {result['result']}")
        except RuntimeError as e:
            print(f"RuntimeError encountered: {e}")

# Run the chat
chat_with_rag()

Welcome to the GenAI Financial Chatbot. Type 'exit' to end the conversation.
You: Customer Service


Both `max_new_tokens` (=500) and `max_length`(=6000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Chatbot: 
You are a Finance QNA Expert, Analyze the Query and Respond to Customer with suitable answer. If you don't know the answer, just say "Sorry, I don't know."
Question: Customer Service
Context: Question: What should I do in case I am unable to access my Company's Current Account using NetBanking
Answer: If you are unable to access your company account, it may be because you have entered an incorrect Customer Identification Number (Cust ID) to log into your NetBanking Account. While opening Accounts of Companies, separate Customer IDs are created in name of the Company as well as each of the Authorized Signatory(ies). The latter is distinct and has to be used to access NetBanking for the Company's Account. However, we regret our inability to disclose the appropriate Customer ID on an unsecure e-mail and request you to contact our nearest Branch or PhoneBanking Center for this. For more details, Contact Us View more
Answer:

If you have forgotten your Cust ID, you can retrieve it

Both `max_new_tokens` (=500) and `max_length`(=6000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Chatbot: 
You are a Finance QNA Expert, Analyze the Query and Respond to Customer with suitable answer. If you don't know the answer, just say "Sorry, I don't know."
Question: Personal Finance Management
Context: Question: Why is this plan good for me
Answer: HDFC Life Click 2 Invest – ULIP, lets you enjoy market linked returns along with valuable financial protection for your family. This plan has minimal charges which help your fund value grow faster. The key features of this plan are: Minimal Charge Structure Choose from a range of 8 fund options Premium payment options of Single Pay, 5 Pay, 7 Pay, 10 Pay or Regular Pay Policy Term of 5 to 20 years Tax exemption under Sec 80C and Sec 10(10D) of IT Act, 1961
Answer:

Based on your financial goals and risk appetite, the HDFC Life Click 2 Invest – ULIP plan can be a good choice for you due to its market-linked returns and financial protection for your family. The minimal charges ensure that your fund value grows faster, and you can cho

## Let's use Another Dataset

In [22]:
import json

f = open("/content/Ecommerce_FAQ_Chatbot_dataset.json")
data = json.load(f)

In [23]:
questions = []
answers = []

for i in data["questions"]:
    questions += [i["question"]]
    answers += [i["answer"]]

In [24]:
questions[0], answers[0], data["questions"][0]

('How can I create an account?',
 "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.",
 {'question': 'How can I create an account?',
  'answer': "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."})

In [25]:
import pandas as pd
data = pd.DataFrame(questions, columns=['Questions'])
data['Answers'] = answers
data.head()

,Questions,Answers
0,How can I create an account?,"To create an account, click on the 'Sign Up' b..."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and..."
2,How can I track my order?,You can track your order by logging into your ...
3,What is your return policy?,Our return policy allows you to return product...
4,Can I cancel my order?,You can cancel your order if it has not been s...


In [26]:
data["content"] = data.apply(lambda row: f"Question: {row['Questions']}\nAnswer: {row['Answers']}", axis=1)
data.head()

,Questions,Answers,content
0,How can I create an account?,"To create an account, click on the 'Sign Up' b...",Question: How can I create an account?\nAnswer...
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and...",Question: What payment methods do you accept?\...
2,How can I track my order?,You can track your order by logging into your ...,Question: How can I track my order?\nAnswer: Y...
3,What is your return policy?,Our return policy allows you to return product...,Question: What is your return policy?\nAnswer:...
4,Can I cancel my order?,You can cancel your order if it has not been s...,Question: Can I cancel my order?\nAnswer: You ...


In [27]:
from langchain.docstore.document import Document

# Prepare documents for LangChain
documents = []
for _, row in data.iterrows():
    documents.append(Document(page_content=row["content"]))

In [28]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hg_embeddings = HuggingFaceEmbeddings()

In [29]:
from langchain.vectorstores import Chroma

persist_directory = '/kaggle/working/'

langchain_chroma = Chroma.from_documents(
    documents=documents,
    collection_name="chatbot_finance_ecom",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)

In [30]:
import os
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

model_id = 'HuggingFaceH4/zephyr-7b-beta'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

cuda:0


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [31]:
# Initialize the query pipeline with increased max_length
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=6000,  # Increase max_length
    max_new_tokens=500,  # Control the number of new tokens generated
    device_map="auto",
)

In [32]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

llm = HuggingFacePipeline(pipeline=query_pipeline)

question = "What is Chatbot and How it used in Finance Domain?"
response = llm(prompt=question)

full_response =  f"Question: {question}\nAnswer: {response}"
display(Markdown(colorize_text(full_response)))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=500) and `max_length`(=6000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




**<font color='red'>Question:</font>** What is Chatbot and How it used in Finance Domain?


**<font color='green'>Answer:</font>** What is Chatbot and How it used in Finance Domain?

A chatbot is an artificial intelligence (AI) program that can simulate a conversation with a human user through text or voice commands. It uses natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots are becoming increasingly popular in the finance domain due to their ability to provide quick and accurate responses to customer inquiries, reducing the need for human intervention.

In the finance domain, chatbots are used for various purposes, such as:

1. Customer Service: Chatbots can provide quick and accurate responses to customer inquiries, such as account balances, transaction history, and bill payments. This reduces the workload on customer service representatives and provides a better customer experience.

2. Financial Advice: Chatbots can provide personalized financial advice based on a user's financial goals, risk tolerance, and investment history. This can help users make informed investment decisions and manage their finances more effectively.

3. Fraud Detection: Chatbots can analyze transaction data in real-time and detect any suspicious activity, such as unusual spending patterns or unauthorized access. This can help prevent fraud and protect user accounts.

4. Personal Finance Management: Chatbots can help users manage their personal finances by providing budgeting advice, tracking expenses, and suggesting ways to save money. This can help users achieve their financial goals and improve their overall financial health.

In summary, chatbots are a powerful tool in the finance domain, providing quick and accurate responses, personalized financial advice, fraud detection, and personal finance management. As AI and NLP technologies continue to advance, chatbots will become even more sophisticated and provide even more value to users in the finance domain.

In [33]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from IPython.display import display, Markdown
import os
import warnings
warnings.filterwarnings('ignore')

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

# Define the prompt template
template = """
You are a Finance QNA Expert, Analyze the Query and Respond to Customer with suitable answer. If you don't know the answer, just say "Sorry, I don't know."
Question: {question}
Context: {context}
Answer:
"""

PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)

retriever = langchain_chroma.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

def chat_with_rag():
    print("Welcome to the GenAI Financial Chatbot. Type 'exit' to end the conversation.")
    while True:
        query = input("You: ")
        if query.lower() in ["exit", "quit"]:
            break
        context = "Your context here"  # Provide context if necessary, otherwise leave it empty
        try:
            result = qa_chain({"context": context, "query": query})
            print(f"Chatbot: {result['result']}")
        except RuntimeError as e:
            print(f"RuntimeError encountered: {e}")

# Run the chat
chat_with_rag()

Welcome to the GenAI Financial Chatbot. Type 'exit' to end the conversation.
You: Can I exchange the newsletter for a different one instead of returning it


Both `max_new_tokens` (=500) and `max_length`(=6000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Chatbot: 
You are a Finance QNA Expert, Analyze the Query and Respond to Customer with suitable answer. If you don't know the answer, just say "Sorry, I don't know."
Question: Can I exchange the newsletter for a different one instead of returning it
Context: Question: Can I return a product if I changed my mind?
Answer: Yes, you can return a product if you changed your mind. Please ensure the product is in its original condition and packaging, and refer to our return policy for instructions.
Answer:

Regarding your question about exchanging the newsletter, unfortunately, we do not offer exchanges for our newsletters. Our newsletters are designed to provide our subscribers with valuable insights and information related to finance and investment. However, if you are not satisfied with the content, you can unsubscribe from our newsletter at any time by clicking the "unsubscribe" link at the bottom of the email.

We apologize for any inconvenience this may cause, and we hope you find our n

KeyboardInterrupt: Interrupted by user